In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib
import warnings
warnings.filterwarnings("ignore")


def load_model(path=''):
  return joblib.load(path)

In [ ]:
lgbm_model=load_model(r"/content/drive/MyDrive/CSE-400(Project Thesis)/lgbm_file_hash.pkl")

In [ ]:
!pip install bnlp-toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 7.0 MB/s 
     |████████████████████████████████| 23.9 MB 53.3 MB/s 
     |████████████████████████████████| 965 kB 58.0 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import numpy as np
import re
from bnlp.corpus import stopwords
from sklearn.feature_extraction.text import HashingVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import *

%matplotlib inline
sns.set()

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
label_auth=pd.read_csv(r'/content/drive/MyDrive/AutoEncoders/LabeledAuthentic-7K.csv') #, nrows=3067)
label_fake=pd.read_csv(r'/content/drive/MyDrive/AutoEncoders/LabeledFake-1K.csv') #, nrows=995)
label_auth.drop(['source','relation'], axis=1, inplace=True)
label_fake.drop(['source','relation','F-type'], axis=1, inplace=True)
df=pd.concat([label_auth, label_fake], axis=0)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df['label']=np.where(df['label']==0.0, 0, 1)
df.tail()
df=df.sample(frac=1).reset_index(drop=True)
print(df.shape)
df.head()

(8501, 7)


,articleID,domain,date,category,headline,content,label
0,2806,banglatribune.com,2018-09-22 19:50:14,Politics,বৃহত্তর ঐক্যের পরীক্ষায় বিএনপি,আগামী নির্বাচনকে সামনে রেখে বৃহত্তর রাজনৈতিক ঐ...,1
1,6888,jagonews24.com,2018-09-21 21:24:28,National,বেনাপোলে ৪৭ হাজার ভারতীয় যৌন উত্তেজক ট্যাবলেট ...,বেনাপোলের পুটখালী সীমান্ত থেকে ১১০ পিস ভারতীয় ...,1
2,5148,channelionline.com,2018-09-22 21:05:41,Sports,পাকিস্তান-আফগানিস্তানের তিন ক্রিকেটারের জরিমানা,আইসিসির আচরণবিধি লঙ্ঘনের দায়ে পাকিস্তান ও আফগা...,1
3,5352,samakal.com,2018-09-20 16:47:40,Politics,অন্তর্জ্বালা থেকে মনগড়া কথা বলেছেন সিনহা: ওবায়...,﻿সাবেক হওয়ার অন্তর্জ্বালা থেকেই বিদেশে বসে সাব...,1
4,7681,jagonews24.com,2018-09-25 16:36:26,Crime,বিশেষায়িত হাসপাতালে চিকিৎসার শুনানি ১ অক্টোবর,দেশের বিশেষায়িত কোনো হাসপাতালে কারাবন্দি বিএনপ...,1


In [ ]:
print('The input text is:\n')
df['content'][2]

The input text is:



'আইসিসির আচরণবিধি লঙ্ঘনের দায়ে পাকিস্তান ও আফগানিস্তানের তিনজন ক্রিকেটারকে জরিমানা করা হয়েছে। শুক্রবার আবুধাবিতে এশিয়া কাপের সুপার ফোরের ম্যাচে ভিন্ন ভিন্ন ঘটনায় অভিযুক্ত হন তারা। পাকিস্তানের হাসান আলী, আফগানিস্তানের অধিনায়ক আসগর আফগান ও স্পিনার রশিদ খানকে ম্যাচ ফি’র ১৫% জরিমানা করে আইসিসি। শুধু আর্থিক জরিমানাই নয়, তাদের প্রত্যেকের নামের পাশে একটি করে ডিমেরিট পয়েন্টও যোগ হয়েছে। আফগান খেলোয়াড়দের জন্য ডিমেরিট পাওয়ার এটি দ্বিতীয় ঘটনা। গত ফেব্রুয়ারিতে প্রথম কোনো আফগান ক্রিকেটার ডিমেরিট পয়েন্ট পেয়েছিলেন। হাসান এবং আসগর আইসিসির আচরণবিধির ২.১.১ ধারা লঙ্ঘন করেছেন। এই ধারায় ক্রিকেটের স্পিরিটের কথা বলা আছে। আর রশিদ খান লঙ্ঘন করেছেন, ২.১.৭ ধারা। যেখানে ভাষা, কাজ বা অঙ্গভঙ্গি ব্যবহার, আন্তর্জাতিক ম্যাচের সময় ব্যাটসম্যানের উপর কোনো বোলারের বা কোনো বোলারের উপর ব্যাটসম্যানের আক্রমণাত্মক প্রতিক্রিয়া সৃষ্টির কথা বলা আছে। ম্যাচের ৩৩তম ওভারের সময় আফগান ব্যাটসম্যান হাসমতউল্লাহ ক্রিজে ফিরে যাওয়ার পরও তার দিকে বল থ্রো করতে যান হাসান আলী। শুধু তাই নয়, পরে ওই ব্যাটসম্যানের দিকে আক্রমণাত্মকভাবে এগিয়ে যান তি

In [ ]:
def preprocess(data):
    """This function is for preprocessing of the news contents. It removes punctuations, English characters and both of Bangla
    and English numerals. It tokenizes all the words and also removes stopwords.
    
    Args: 
        The Entire Dataframe
    Returns:
        Preprocessed news corpuses in a list of lists
    """
    corpus=[]
    for i in range(0, len(data)):
        x=re.sub('[^\u0980-\u09FF]',' ',data['content'][i])
        x=re.sub('[a-zA-Z0-9]+', ' ', x)
        x=re.sub('[০১২৩৪৫৬৭৮৯]+', ' ', x)
        x=x.split()
        x=[w for w in x if w not in set(stopwords)]
        x=' '.join(x)
        corpus.append(x)
    
    return corpus


corpus=preprocess(df)
corpus[16:20]

['মার্কিন সংবাদমাধ্যম এবিসি গুড মর্নিং আমেরিকা অনুষ্ঠানে কুক বানানো সবচেয়ে উন্নত আইফোন ভোক্তাদের প্রয়োজনীয় গ্যাজেটের প্রয়োজনীয়তা মেটানো সম্ভব বলেও ভাষ্য কুক আইফোন ডিজিটাল ক্যামেরার জায়গা নিয়েছে আলাদা ক্যামেরা দরকার ভিডিও ক্যামেরার জায়গা নিয়েছে মিউজিক প্লেয়ারের জায়গা নিয়েছে ভিন্ন ডিভাইসের জায়গা নিয়েছে তর্কসাপেক্ষে পণ্য সত্যিই গুরুত্বপূর্ণ দেখেছি মানুষ সবচেয়ে উদ্ভাবনী পণ্যটি পেতে চায় সস্তায় যায় আইফোন ম্যাক্স দাম যথাক্রমে ডলার সর্বোচ্চ মূল্য ডলার ম্যাক কম্পিউটারের চেয়েও কুক এগুলোর অধিকাংশ মানুষের উপায় ক্যারিয়ারের টেলিযোগাযোগ নেটওয়ার্ক চুক্তি মাসিক কিস্তিতে অর্থ পরিশোধ ডলারের দামের আইফোনের দিকেও দেখেন অধিকাংশ মানুষ মাসে ডলার পরিশোধ হিসেবে প্রতিদিন এক ডলার ডেওয়া লাগবে',
 'শাহজালাল বিজ্ঞান প্রযুক্তি বিশ্ববিদ্যালয়ের শাবি ছাত্রী চুরির ঘটনায় তিন সদস্যের তদন্ত কমিটি গঠন হয়েছে বৃহস্পতিবার সেপ্টেম্বর দুপুরে কমিটি গঠন ছাত্রী হলের প্রাধ্যক্ষ অধ্যাপক আমিনা পারভীন খবর নিশ্চিত প্রাধ্যক্ষ অধ্যাপক আমিনা পারভীন বিশ্ববিদ্যালয়ের ছাত্রী চুরির ঘটনা তদন্তে স্কুল অব এগ্রিকালচার অ্যান্ড মিনারেল সায়েন্সের ডিন অধ্

In [ ]:
print('The cleaned text is:\n')
corpus[2]

The cleaned text is:



'আইসিসির আচরণবিধি লঙ্ঘনের দায়ে পাকিস্তান আফগানিস্তানের তিনজন ক্রিকেটারকে জরিমানা হয়েছে শুক্রবার আবুধাবিতে এশিয়া কাপের সুপার ফোরের ম্যাচে ভিন্ন ভিন্ন ঘটনায় অভিযুক্ত পাকিস্তানের হাসান আলী আফগানিস্তানের অধিনায়ক আসগর আফগান স্পিনার রশিদ খানকে ম্যাচ ফি জরিমানা আইসিসি আর্থিক জরিমানাই প্রত্যেকের নামের পাশে ডিমেরিট পয়েন্টও যোগ হয়েছে আফগান খেলোয়াড়দের ডিমেরিট পাওয়ার দ্বিতীয় ঘটনা গত ফেব্রুয়ারিতে আফগান ক্রিকেটার ডিমেরিট পয়েন্ট পেয়েছিলেন হাসান আসগর আইসিসির আচরণবিধির ধারা লঙ্ঘন ধারায় ক্রিকেটের স্পিরিটের কথা রশিদ খান লঙ্ঘন ধারা ভাষা অঙ্গভঙ্গি আন্তর্জাতিক ম্যাচের সময় ব্যাটসম্যানের বোলারের বোলারের ব্যাটসম্যানের আক্রমণাত্মক প্রতিক্রিয়া সৃষ্টির কথা ম্যাচের তম ওভারের সময় আফগান ব্যাটসম্যান হাসমতউল্লাহ ক্রিজে যাওয়ার পরও বল থ্রো হাসান আলী ব্যাটসম্যানের আক্রমণাত্মকভাবে এগিয়ে হাসান আলীর সঙ্গেই তর্কে জড়িয়ে আফগান অধিনায়ক আসগর আফগান ম্যাচের তম ওভারে পাকিস্তানি ব্যাটসম্যান আসিফ আলীকে আউট আক্রমণাত্মক অঙ্গভঙ্গি রশিদ খান ব্যাটসম্যানের আঙুল সাজঘরের পথও দেখান ম্যাচের তিন ক্রিকেটারকেই শুনানিতে ডেকেছিলেন ম্যাচ রেফারি অ

In [ ]:
hashing=HashingVectorizer(n_features=5000, ngram_range=(1, 3))
X=hashing.fit_transform(corpus).toarray()
y=df['label'].values
X.shape, y.shape

((8501, 5000), (8501,))

In [ ]:
print('The input vector after hashing is:\n')
print(X[2][0:30])

The input vector after hashing is:

[ 0.          0.          0.          0.          0.          0.
  0.         -0.13130643  0.         -0.04376881  0.          0.
  0.          0.          0.          0.         -0.08753762  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]


In [ ]:
lgbm_pred=lgbm_model.predict(X)

y=['Fake' if x==0 else 'Real' for x in y]
lgbm_pred=['Fake' if x==0 else 'Real' for x in lgbm_pred]

print(f'Actual Outcome ---> {y[2]} \n')
print(f'Prediction Outcome ---> {lgbm_pred[2]}')

Actual Outcome ---> Real 

Prediction Outcome ---> Real


In [ ]:
df[df['label'] == 0].head()

,articleID,domain,date,category,headline,content,label
5,1202,earki.com,"২১:২৯, জানুয়ারি ১৮, ২০১৯",Politics,কোনো কিছু বলছ না? ও ওমর সানী!,ওমর সানীকে ডেকে বলে মৌসুমীআমার বিপদে কিছু বলছো...,0
7,329,earki.com,"২০:৪৩, জানুয়ারি ২৬, ২০১৯",Miscellaneous,স্বামী বিয়ের রাতে স্ত্রীর যে অজানা নিয়মটি জানত...,এক নব দম্পতির বিয়ের রাতে জামাই বৌকে বলছে-‘আমি...,0
10,659,bengalbeats.com,2019-06-21T17:22:54+00:00,Miscellaneous,তীব্র গরমে টয়লেটের পর ট্যাপের পানিতে পশ্চাৎদেশ...,তীব্র গরমে টয়লেটের পর ট্যাপের পানি দিয়ে শুচু ক...,0
26,216,channeldhaka.news,2017-09-13T11:08:46+00:00,Miscellaneous,রোহিঙ্গাদের বিষয়ে আল্লাহর সাথে সরাসরি আলাপ হচ্...,"নাম মাহবুব এ খোদা, সর্বস্তরে দেওয়ানবাগী হুজুর...",0
29,1242,channeldhaka.news,2018-09-14T19:08:41+00:00,National,নোয়াখালী ছাড়া দেশের কোথাও ফকির মিসকিন নেই,সম্প্রতি ফেসবুকে ভাইরাল হওয়া বিটিভির এক বিজ্ঞা...,0


In [ ]:
print('The input text is:\n')
df['content'][29]

The input text is:



'সম্প্রতি ফেসবুকে ভাইরাল হওয়া বিটিভির এক বিজ্ঞাপনের ভিত্তিতে ছোট্ট একটি জরীপ চালায় চ্যানেল ঢাকার প্রতিনিধি দল। জরীপে বেড়িয়ে আসে কিছু ভয়ংকর সত্য। উল্লেখ্য যে, বিটিভির ওই শর্টফিল্মটি নোয়াখালী তে শ্যুট করা হয়েছিলো।এ থেকে বুঝে নেয়া যায়, বর্তমান সরকারের উন্নয়নমূলক বিভিন্ন কর্মকান্ডের ফলে সমগ্র দেশ হতে ফকির-মিসকিন দূর হলেও স্বভাবগত কারণে নোয়াখালী থেকে ফকির মিসকিন দূরীকরণ সম্ভব হয়নি।জানা যায়, দারিদ্রতা নয় বরং পারিবারিক ঐতিহ্য ও সংস্কৃতি বজায় রাখার জন্য ভিক্ষাবৃত্তি থেকে নিজেদের দূরে রাখতে পারছেন না নোয়াখালী বাসী এবং বিষয়টিকে তারা গর্বের সাথেই দেখেন। এটাকে তারা পারিবারিক ব্যবসা হিসেবেই দেখছেন। বিশ্বের মোট ফকির মিসকিন ও ভিখারীর প্রায় পোছাত্তর শতাংশই নোয়াখালী’র।'

In [ ]:
print('The cleaned text is:\n')
corpus[29]

The cleaned text is:



'ফেসবুকে ভাইরাল হওয়া বিটিভির এক বিজ্ঞাপনের ভিত্তিতে ছোট্ট জরীপ চালায় চ্যানেল ঢাকার প্রতিনিধি দল জরীপে বেড়িয়ে আসে ভয়ংকর সত্য উল্লেখ্য বিটিভির শর্টফিল্মটি নোয়াখালী তে শ্যুট হয়েছিলো বুঝে নেয়া যায় বর্তমান সরকারের উন্নয়নমূলক কর্মকান্ডের সমগ্র দেশ ফকির মিসকিন দূর স্বভাবগত কারণে নোয়াখালী ফকির মিসকিন দূরীকরণ সম্ভব হয়নি যায় দারিদ্রতা পারিবারিক ঐতিহ্য সংস্কৃতি বজায় রাখার ভিক্ষাবৃত্তি দূরে রাখতে পারছেন নোয়াখালী বাসী বিষয়টিকে গর্বের সাথেই দেখেন এটাকে পারিবারিক ব্যবসা হিসেবেই দেখছেন বিশ্বের ফকির মিসকিন ভিখারীর পোছাত্তর শতাংশই নোয়াখালী'

In [ ]:
print('The input vector after hashing is:\n')
print(X[29][0:30])

The input vector after hashing is:

[ 0.          0.          0.          0.          0.          0.
  0.         -0.15249857  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.07624929  0.        ]


In [ ]:
print(f'Actual Outcome ---> {y[29]} \n')
print(f'Prediction Outcome ---> {lgbm_pred[29]}')

Actual Outcome ---> Fake 

Prediction Outcome ---> Fake
